In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
import networkx as nx
import scipy.spatial.distance

In [ ]:
df = pd.read_csv('Full_data.csv',encoding='cp1252')

In [ ]:
df.head(10)

,id,skills,titles,major
0,/in/cassandra-vose/,"['Data Warehousing', 'Data Analysis', 'Busines...","['Senior Manager/Data Engineer', 'Consultant']","['Bachelor of Science, Civil & Environmental E..."
1,/in/samitra-harrington-3ab715141/,1,['Research Ambassador'],"[""Bachelor's degree, Sociology""]"
2,/in/kavyadagli/,"['Python', 'Java', 'JavaScript', 'C', 'HTML', ...","['Software Engineer Intern', 'Teaching Assista...","['Bachelor of Science - BS, Computer Science']"
3,/in/melvin-molina/,"['Java', 'C (Programming Language)', 'Python (...","['Associate Software Engineer', 'High School T...","['Bachelor of Science - BS, Computer Science',..."
4,/in/sarah-nguyen-4706b4205/,['NaN'],['NaN'],"['Bachelor of Science - BS, Community Health a..."
5,/in/shivppatel/,"['Java', 'Python', 'JavaScript', 'Git', 'Probl...","['University of Maryland', 'University of Mary...",['Bachelor of Science - BS']
6,/in/excela/,"['Leadership', 'Research', 'Microsoft Office',...","['Software Automation Engineer (Intern)', 'Und...","['Bachelor of Engineering - BE, Computer Engin..."
7,/in/adam-jacques/,"['SolidWorks', 'Python (Programming Language)'...","['University Recreation and Wellness', 'Univer...","['Bachelor of Science - BS, Mechanical Enginee..."
8,/in/azariah-knox-529a95170/,['NaN'],['NaN'],"['Bachelor of Science - BS, Mechanical Enginee..."
9,/in/clydeapaid/,"['Microsoft Office', 'Financial Analysis', 'Ma...","['Manager', 'Pathways Operations Management In...","['Master of Business Administration (M.B.A.)',..."


In [ ]:
#Remove the rows that empty rows
df = df[df['skills'] != "['NaN']"]	
df = df[df['titles'] != "['NaN']"]	
df = df[df['major'] != "['NaN']"]	
#Remove the rows that empty rows
df = df[df.skills.apply(lambda x: not(x.isnumeric()))]
df = df[df.titles.apply(lambda x: not(x.isnumeric()))]
df = df[df.major.apply(lambda x: not(x.isnumeric()))]
#Converts string version of lists into lists 
df['skills'] = df.skills.apply(lambda x: literal_eval(str(x)))
df['titles'] = df.titles.apply(lambda x: literal_eval(str(x)))
df['major'] = df.major.apply(lambda x: literal_eval(str(x)))
#Resets row counter
df = df.reset_index(drop=True)

print(type(df['major'][0]))
print(type(df['titles'][0]))
print(type(df['skills'][0]))


<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
total = []
indexcount = []
for index, row in df.iterrows():
    cur_title = []
    for title in row['titles']:
        cur_title.append(title.rsplit('<span class')[0])
    cur_title = list(set(cur_title))
    df.loc[index]['titles'] = cur_title
    cur_major = ''
    for major in row['major']:
        string_major = major.rsplit('<span class')[0]
        if 'Bachelor' in string_major:
            cur_major = string_major 
            continue
    if not cur_major:
        indexcount.append(index)
    df.loc[index]['major'] = cur_major
    cur_skill = []
    for skill in row['skills']:
        cur_skill.append(skill.rsplit('<span class')[0])
    cur_skill = list(set(cur_skill))
    df.loc[index]['skills'] = cur_skill
print(indexcount)
df = df.drop(indexcount)
df = df.reset_index(drop=True)

[15, 22, 23, 24, 29, 31, 33, 36, 48, 55, 60, 62, 73, 80, 83, 85, 89, 92, 94, 97, 99, 108, 111, 115, 119, 126, 127, 128]


In [ ]:
df_origin = df.copy()
df.head()

,id,skills,titles,major
0,/in/cassandra-vose/,"[Data Warehousing, Analysis, Leadership, SQL, ...","[Consultant, Senior Manager/Data Engineer]","Bachelor of Science, Civil & Environmental Eng..."
1,/in/kavyadagli/,"[HTML, C, JavaScript, Java, Python]",[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science"
2,/in/melvin-molina/,"[C (Programming Language), Java, Python (Progr...","[Software Engineer, Technical Researcher, Inst...","Bachelor of Science - BS, Computer Science"
3,/in/shivppatel/,"[Git, JavaScript, Java, Communication, Problem...",[University of Maryland],Bachelor of Science - BS
4,/in/excela/,"[Tutoring, English, Leadership, Problem Solvin...","[Software Automation Engineer (Intern), Studen...","Bachelor of Engineering - BE, Computer Enginee..."


In [ ]:
total = []
for index, row in df.iterrows():
    for skill in row['skills']:
        total.append(skill)
#print(len(total))
print('Number of different skills:',len(list(set(total))))
unique_skills = list(set(total))


Number of different skills: 522


In [ ]:
unique_majors = list(set(df['major'].values.tolist()))
len(unique_majors)

89

Graph

In [ ]:
major_graph = nx.Graph() 
for major in unique_majors:
    for index, row in df.iterrows():
        if row['major'] == major:
            for cur_title in row['titles']:
                major_graph.add_edge(major,cur_title)
nx.write_gexf(major_graph, "majors.gexf")                
                

In [ ]:
skill_graph = nx.Graph() 
for uniskil in unique_skills:
    for index, row in df.iterrows():
        if uniskil in row['skills']:
            for job in row['titles']:
                skill_graph.add_edge(uniskil,job)
nx.write_gexf(skill_graph, "jobs.gexf")         

Similarity Analysis


In [ ]:

skill_df = pd.DataFrame
mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('skills')),
                index=df.index,
                columns=mlb.classes_))

In [ ]:
df.shape

(112, 525)

In [ ]:
df.head()

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
0,/in/cassandra-vose/,"[Consultant, Senior Manager/Data Engineer]","Bachelor of Science, Civil & Environmental Eng...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,/in/kavyadagli/,[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,/in/melvin-molina/,"[Software Engineer, Technical Researcher, Inst...","Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/in/shivppatel/,[University of Maryland],Bachelor of Science - BS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,/in/excela/,"[Software Automation Engineer (Intern), Studen...","Bachelor of Engineering - BE, Computer Enginee...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = df.iloc[:,3:,]
data.head()

,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,Adobe Creative Suite,Adobe Illustrator,Adobe Indesign CC,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
arr = np.asarray(data)
arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
target = df[df['id'] == '/in/kavyadagli/'].iloc[:,3:,]
target = np.asarray(target)
print(target)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
distances = scipy.spatial.distance.cdist(arr,target,'cosine')[:,0]

query_distances = list(zip(df.id, distances))
print(query_distances)

[(0, 1.0), (1, 2.220446049250313e-16), (2, 0.7418011102528389), (3, 0.4522774424948339), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 0.8), (11, 1.0), (12, 1.0), (13, 1.0), (14, 0.8709005551264195), (15, 0.49290744716289014), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 0.7418011102528389), (22, 1.0), (23, 0.850928801500014), (24, 1.0), (25, 0.7763932022500211), (26, 1.0), (27, 1.0), (28, 0.5527864045000421), (29, 1.0), (30, 0.9105572809000084), (31, 1.0), (32, 1.0), (33, 1.0), (34, 0.7763932022500211), (35, 1.0), (36, 1.0), (37, 0.7763932022500211), (38, 1.0), (39, 1.0), (40, 0.683772233983162), (41, 0.683772233983162), (42, 1.0), (43, 1.0), (44, 0.7763932022500211), (45, 1.0), (46, 1.0), (47, 1.0), (48, 1.0), (49, 1.0), (50, 1.0), (51, 1.0), (52, 0.7418011102528389), (53, 1.0), (54, 0.6348516283298893), (55, 1.0), (56, 1.0), (57, 0.7763932022500211), (58, 0.6348516283298893), (59, 0.8651600275073515), (60, 1.0), (61, 1.0), (62, 0.683772233983162), (

In [ ]:
sorted_by_second = sorted(query_distances, key=lambda tup: tup[1])
sorted_by_second

In [ ]:
found = df[df['id']=='/in/linda-johnson-mba-5a2a2a64/']
original = df[df['id'] == '/in/kavyadagli/']
left = []
right = []
for i in df.columns:
    if found[i].values[0] == 1:
        left.append(i)
    if original[i].values[0] == 1:
        right.append(i)
mid = set(left).union(set(right))
print(len(mid))
print(mid)


39
{'CATIA', 'Team Leadership', 'C', 'JavaScript', 'Leadership', 'Public Speaking', 'Microsoft Office', 'Microsoft Project', 'Matlab', 'Data Analysis', 'Management', 'Kaizen Leadership', 'Lean Six Sigma', 'Microsoft PowerPoint', 'Time Management', 'Java', 'Mechanical Engineering', 'Problem Solving', 'Manufacturing', 'Research', 'Oracle Database', 'PowerPoint', 'Teamwork', 'Event Planning', 'SolidWorks', 'Engineering', 'Microsoft Excel', 'WMS Implementations', 'Microsoft Word', 'PTS', 'Project Management', 'HTML', 'Customer Service', 'English', 'Cross-functional Team Leadership', 'Social Media', 'Windows', 'Python', 'WMS (Warehouse Management System)'}


In [ ]:
found[i].values[0]

0

In [ ]:
found

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
20,/in/linda-johnson-mba-5a2a2a64/,"[Manufacturing Operations Supervisor, Undergra...","Bachelor of Science (BS), Mechanical Engineering",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
original

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
1,/in/kavyadagli/,[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_origin[df_origin['id'] == '/in/linda-johnson-mba-5a2a2a64/']['titles'].values


array([list(['Manufacturing Operations Supervisor', 'Undergraduate Research Assistant', 'Associate Supply Chain Engineer', 'Corporate Relations Board Chair', 'Conference Planning Chair', 'Mechanical Engineering Intern, Global Engineering Service Center', 'How Stuff Works Teaching Assistant', 'Program Manager', 'Intern for Cellulose Esters and Specialty Plastics Department', 'NPI OLM Program Manager', 'Mechanical Engineering Intern'])],
      dtype=object)